Q1: Running Ollama with Docker

In [4]:
!docker exec -it ollama bash -c "ollama -v"

ollama version is 0.2.1


In [11]:
!sudo docker exec -it ollama ls -l /root/.ollama/models/manifests/registry.ollama.ai/library/gemma/

total 4
-rw-r--r-- 1 root root 856 Jul 10 03:01 2b


In [12]:
!sudo docker exec -it ollama cat /root/.ollama/models/manifests/registry.ollama.ai/library/gemma/2b

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}

In [22]:
!sudo docker exec -it ollama ollama run gemma:2b "10 * 10 ="

⠙ ⠹ ⠹ ⠸ ⠼ ⠴ ⠧ ⠧ Sure, here's the answer to 10 * 10:

```
10 * 10 = 100
```

The answer to 10 * 10 is 100.



In [25]:
!docker exec -it ollama ollama pull gemma:2b

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [3]:
import os
import subprocess
import time

def run_command(command, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE):
    """
    Runs a shell command and captures the output.
    Args:
        command (str): The shell command to run.
        check (bool): Whether to raise an error on failure.
        stdout: Standard output pipe.
        stderr: Standard error pipe.
    Returns:
        str: The standard output from the command.
    """
    process = subprocess.Popen(command, shell=True, stdout=stdout, stderr=stderr)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        error_message = f"Error running command: {command}\n{stderr.decode('utf-8')}"
        if check:
            raise RuntimeError(error_message)
        print(error_message)
    return stdout.decode('utf-8')

# 1. Create a directory for ollama files
print("Creating directory for ollama files...")
os.makedirs('ollama_files', exist_ok=True)

# 2. Run the container and pull the weights
print("Running the container and pulling the weights...")
run_command("docker run -d --rm -v $(pwd)/ollama_files:/root/.ollama -p 11434:11434 --name ollama ollama/ollama")

# 3. Wait for the container to start
print("Waiting for the container to start...")
time.sleep(10)  # Increase the wait time if needed

# 4. Pull the weights inside the running container
print("Pulling the weights inside the running container...")
run_command("docker exec ollama ollama pull gemma:2b", check=True)

# 5. Stop the container
print("Stopping the container...")
run_command("docker stop ollama", check=True)

# 6. Create a Dockerfile
print("Creating Dockerfile...")
dockerfile_content = """
FROM ollama/ollama

COPY ./ollama_files /root/.ollama

WORKDIR /root/.ollama
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)

# 7. Build the new Docker image
print("Building the new Docker image...")
run_command("docker build -t ollama-with-weights .", check=True)

# 8. Run the new Docker image
print("Running the new Docker image...")
run_command("docker run -d --rm -v $(pwd)/ollama_files:/root/.ollama -p 11434:11434 --name ollama ollama-with-weights", check=True)

print("Process completed successfully.")

Creating directory for ollama files...
Running the container and pulling the weights...
Waiting for the container to start...
Pulling the weights inside the running container...
Stopping the container...
Creating Dockerfile...
Building the new Docker image...
Running the new Docker image...
Process completed successfully.


In [3]:
! pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 839.4 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 6.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 7.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.35.12
    Uninstalling openai-1.35.12:
      Successfully uninstalled openai-1.35.12

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
